In [2]:
from discussion_agents.cog.prompts.self_refine import (
    GSM8K_FEEDBACK_FEWSHOT_EXAMPLES,
    GSM8K_FEWSHOT_EXAMPLES,
    SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K,
    SELF_REFINE_INSTRUCTION_GSM8K,
)
from discussion_agents.cog.functional.self_refine import _prompt_agent, _prompt_feedback
from langchain_community.chat_models.openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=openai_api_key)

question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"

In [4]:
examples = GSM8K_FEWSHOT_EXAMPLES
prompt = SELF_REFINE_INSTRUCTION_GSM8K

solution = _prompt_agent(
    llm=llm,
    question=question,
    examples=examples,
    prompt=prompt
)

<==============================================================>
# Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
# solution using Python:

def solution():
    """Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
    jason_lollipops_initial = 20
    jason_lollipops_after = 12
    denny_lollipops = jason_lollipops_initial - jason_lollipops_after
    result = denny_lollipops
    return result



# Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
# solution using Python:

def solution():
    """There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"""
    trees_initial = 15
    

In [5]:
print(solution)

def solution():
    """A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"""
    blue_fiber = 2
    white_fiber = blue_fiber / 2
    total_fiber = blue_fiber + white_fiber
    result = total_fiber
    return result


In [6]:
feedback_examples = GSM8K_FEEDBACK_FEWSHOT_EXAMPLES
feedback_prompt = SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K

feedback_and_improved_solution = _prompt_feedback(
    llm=llm,
    examples=feedback_examples,
    solution=solution,
    prompt=feedback_prompt
)

<FEEDBACK==============================================================>
def solution():
    """Kelly is grocery shopping at a supermarket and is making sure she has enough in her budget for the items in her cart. Her 5 packs of bacon cost $10 in total and she has 6 packets of chicken which each cost twice as much as a pack of bacon. She also has 3 packs of strawberries, priced at $4 each, and 7 packs of apples, each priced at half the price of a pack of strawberries. If Kelly’s budget is $65 then how much money, in dollars, does she have left in her budget?"""
    budget = 65
    bacon = 5
    bacon_cost = 10
    chicken = 6
    chicken_cost = 2 * bacon_cost
    strawberries = 3
    strawberry_cost = 4
    apples = 7
    apple_cost = strawberry_cost / 2
    total_cost = bacon_cost + chicken_cost + strawberry_cost + apple_cost
    money_left = budget - total_cost
    result = money_left
    return result

# There is an error in the code above because of lack of understanding of the que

In [28]:
feedback_and_improved_solution

'The error in the code is in the calculation of the white fiber needed for the robe. The question states that the robe takes 2 bolts of blue fiber and half that much white fiber. This means that the white fiber needed should be half of the blue fiber needed, not the other way around.\n\nHere is the corrected code:\n\n```python\ndef solution():\n    """A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"""\n    blue_fiber = 2\n    white_fiber = blue_fiber / 2\n    total_fiber = blue_fiber + white_fiber\n    result = total_fiber\n    return result\n```'

In [ ]:
feedback = feedback_and_improved_solution.split("def solution():")[0]
improved_solution = "def solution():" + \
    feedback_and_improved_solution.split("def solution():")[1].split("### END")[0].rstrip()
prefix = f"""{question_prefix}{solution}{intra_example_sep}{feedback_instruction}{answer_prefix}"""
gen_ans = f"""

{feedback}

{improved_solution.rstrip()}{inter_example_sep}"""
new_example = f"{prefix}{gen_ans}"
feedback_examples = f"{feedback_examples}{new_example}"
